In [1]:
import boto3, json

In [3]:
bedrock_runtime = boto3.client(
    service_name = "bedrock-runtime",
    region_name= "us-west-2"
)

In [4]:
prompt = "Hello"

In [10]:
kwargs = { 
"modelId": "ai21.j2-ultra-v1", 
"body": "{\"prompt\":\""+prompt+"\",\"maxTokens\":200,\"temperature\":0.7,\"topP\":1,\"stopSequences\":[],\"countPenalty\":{\"scale\":0},\"presencePenalty\":{\"scale\":0},\"frequencyPenalty\":{\"scale\":0}}",
}

print(kwargs)

{'modelId': 'ai21.j2-ultra-v1', 'body': '{"prompt":"Hello","maxTokens":200,"temperature":0.7,"topP":1,"stopSequences":[],"countPenalty":{"scale":0},"presencePenalty":{"scale":0},"frequencyPenalty":{"scale":0}}'}


In [11]:
response = bedrock_runtime.invoke_model(**kwargs)
response
print("XD")

AccessDeniedException: An error occurred (AccessDeniedException) when calling the InvokeModel operation: You don't have access to the model with the specified model ID.